In [ ]:
from langchain.llms import OpenAI
from langchain.tools import ShellTool
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.callbacks import HumanApprovalCallbackHandler

## Setting the LLM

In [ ]:
llm = OpenAI(
    model_name = "gpt-3.5-turbo-instruct",
    temperature = 0,
    openai_api_key = open("openai_api.txt").read()
)

## Human-in-the-loop Tool Validation

Let's suppose we need to make use of the `ShellTool`.

Adding this tool to an automated flow poses obvious risks. Let's see how we could enforce manual human approval of inputs going into this tool.

In [ ]:
## Initializing the Tool

tool = ShellTool()

In [ ]:
print(tool.run("echo Hello World!"))

In [ ]:
## Adding Human Approval

tool = ShellTool(callbacks=[HumanApprovalCallbackHandler()])

In [ ]:
print(tool.run("ls /usr"))

In [ ]:
print(tool.run("ls /private"))

## Configuring Human Approval

Let's suppose we have an agent that takes in multiple tools, and we want it to only trigger human approval requests on certain tools and certain inputs. We can configure out callback handler to do just this.

In [ ]:
def _should_check(serialized_obj: dict) -> bool:
    # Only require approval on ShellTool.
    return serialized_obj.get("name") == "terminal"


def _approve(_input: str) -> bool:
    if _input == "echo 'Hello World'":
        return True
    msg = (
        "Do you approve of the following input? "
        "Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no."
    )
    msg += "\n\n" + _input + "\n"
    resp = input(msg)
    return resp.lower() in ("yes", "y")


callbacks = [HumanApprovalCallbackHandler(should_check=_should_check, approve=_approve)]

In [ ]:
tools = load_tools(["wikipedia", "llm-math", "terminal"], llm=llm)

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [ ]:
agent.run(
    "It's 2023 now. How many years ago did Konrad Adenauer become Chancellor of Germany.",
    callbacks = callbacks
)

In [ ]:
agent.run(
    "print 'Hello World' in the terminal",
    callbacks = callbacks
)

In [ ]:
agent.run(
    "list all directories in /private",
    callbacks = callbacks
)